In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
from scipy.linalg import  solve
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import os
from tqdm import  tqdm
from joblib import Parallel, delayed
import sys
if sys.version_info[0] < 3:
	raise Exception("Python 3 not detected.")
import matplotlib.pyplot as plt
from scipy import io
os.chdir('data')
fields = "training_data", "training_labels", "test_data"
data ={}
if __name__ == "__main__":
    for data_name in ["mnist", "spam"]:
        data[data_name] = np.load(f"{data_name}-data-hw3.npz")
        print("\nloaded %s data!" % data_name)
        for field in fields:
            print(field, data[data_name][field].shape)
os.chdir('..')

seed = 42
np.random.seed(seed)


loaded mnist data!
training_data (60000, 1, 28, 28)
training_labels (60000,)
test_data (10000, 1, 28, 28)

loaded spam data!
training_data (4171, 32)
training_labels (4171,)
test_data (1000, 32)


In [4]:

train = data["mnist"]["training_data"]
labels = data["mnist"]["training_labels"]
train = train.reshape(train.shape[0], -1)  

normalize = lambda x: x / (np.linalg.norm(x, ord=2, axis=1, keepdims=True) + 1e-8)

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

#basically a baysian inference model
class GaussClassifier:
    def __init__(self):
        self.mus = {}
        self.covs = {}
        self.priors = {}
        self.classes = None
        self.epsilon = 1e-5
        self.cov_calc = {}
    def normalize(self, x):
        norm = np.linalg.norm(x, ord=2, axis=1, keepdims=True)  # Compute L2 norm
        norm = np.where(norm == 0, self.epsilon, norm)  # Replace zeros with small value
        return x / norm
    def fit(self, X, Y, post=True):
        self.post = post
        X = self.normalize(X)
        self.classes = np.unique(Y).tolist()
        for c in self.classes:
            x_c = X[Y == c]
            mu = np.mean(x_c, axis=0)
            cov = np.cov(x_c, bias=False, rowvar=False)
            self.cov_calc[c] = cov
            prior = len(x_c) / Y.shape[0]
            self.mus[c] = mu
            self.covs[c] = cov + self.epsilon * np.eye(cov.shape[0])  # Regularization
            self.priors[c] = prior 
    def probability(self, X):
        X_cpu =X
        mus_cpu = np.stack([self.mus[c] for c in self.classes])
        covs_cpu = {c: self.covs[c] for c in self.classes}
        priors_cpu = np.array([self.priors[c] for c in self.classes])
        log_probs = np.zeros((X_cpu.shape[0], len(self.classes))) 

        for idx, c in tqdm(enumerate(self.classes), desc="Predicting..."):
            log_probs[:, idx] = multivariate_normal.logpdf(X_cpu, mean=mus_cpu[idx], cov=covs_cpu[c])
            if self.post:
                log_probs[:,idx] += np.log(priors_cpu[idx])
        return np.array(self.classes)[np.argmax(np.array(log_probs), axis=1)]


    def predict(self, X):
        X = self.normalize(X)
        return self.probability(X)

model = GaussClassifier()
model.fit(train, labels, False)
preds = model.predict(train)
accuracy = np.mean(preds == labels)
print(f"Train Accuracy: {accuracy}")
print(classification_report(labels, preds))

Predicting...: 10it [00:18,  1.88s/it]


Train Accuracy: 0.92965
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5923
           1       0.97      0.96      0.96      6742
           2       0.94      0.94      0.94      5993
           3       0.93      0.92      0.92      6162
           4       0.97      0.91      0.94      5862
           5       0.97      0.86      0.91      5428
           6       0.97      0.96      0.97      5870
           7       0.96      0.88      0.92      6213
           8       0.81      0.93      0.86      5851
           9       0.84      0.95      0.89      5956

    accuracy                           0.93     60000
   macro avg       0.93      0.93      0.93     60000
weighted avg       0.93      0.93      0.93     60000



In [5]:
class LDAClassifier(GaussClassifier):
    def __init__(self):
        super().__init__()

    def fit(self, X, Y):
        super().fit(X, Y) 
        X = self.normalize(X)
        N = len(X)
        k = len(self.classes)
        self.cov_sum = np.zeros((X.shape[1], X.shape[1]))

        # Compute the pooled covariance matrix
        for c in self.classes:
            X_c = X[Y == c]
            self.cov_sum += len(X_c) * self.covs[c]  # Weighting by class size

        self.cov_sum /= (N - k)  # Normalize pooled covariance
        self.cov_sum_inv = solve(self.cov_sum, np.eye(self.cov_sum.shape[0]), assume_a='sym')  # More stable than np.linalg.inv()
    def probability(self, X):

        scores = np.zeros((X.shape[0], len(self.mus)))
        for idx, c in tqdm(enumerate(self.mus.keys()), total=len(self.mus)):
            mu_c = self.mus[c]
            term1 = X @ self.cov_sum_inv @ mu_c  # x^T Sigma^{-1} mu_C
            term2 = 0.5 * mu_c.T @ self.cov_sum_inv @ mu_c  # (1/2) mu_C^T Sigma^{-1} mu_C
            term3 = np.log(self.priors[c])  # ln(prior)
            scores[:, idx] = term1 - term2 + term3
        return np.argmax(scores, axis=1) 
    def predict(self, X):
        X = self.normalize(X)
        return self.probability(X)
        

model = LDAClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = np.mean(preds == y_test)
print(f"Train Accuracy: {accuracy}")
print(classification_report(y_test, preds))

100%|██████████| 10/10 [00:01<00:00,  7.44it/s]

Train Accuracy: 0.8734166666666666
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1109
           1       0.92      0.95      0.94      1339
           2       0.90      0.83      0.86      1254
           3       0.83      0.85      0.84      1183
           4       0.86      0.90      0.88      1174
           5       0.85      0.81      0.83      1107
           6       0.93      0.92      0.92      1183
           7       0.93      0.85      0.88      1258
           8       0.79      0.81      0.80      1177
           9       0.81      0.87      0.84      1216

    accuracy                           0.87     12000
   macro avg       0.87      0.87      0.87     12000
weighted avg       0.87      0.87      0.87     12000



In [6]:

class QDAClassifier(GaussClassifier):
    def __init__(self,):
        super().__init__()

    def fit(self,X,Y):
        super().fit(X,Y,True)
        X = self.normalize(X)
        self.covs_inv = {}
        for c in self.classes:
            self.covs_inv[c] = solve(self.covs[c], np.eye(self.covs[c].shape[0]), assume_a='sym')  # More stable than np.linalg.inv()

    def probability(self, X):
        scores = np.zeros((X.shape[0], len(self.mus)))
        for idx, c in tqdm(enumerate(self.classes), desc="Calculating probablities"):
            mean_c = self.mus[c]
            cov_c = self.covs[c]

            sign, logdet = np.linalg.slogdet(cov_c)  # Compute log determinant
            inv_cov = np.linalg.inv(cov_c)  # Compute inverse
            # Compute quadratic discriminant function
            for i, x in enumerate(X):
                diff = x - mean_c
                term1 = -0.5 * np.dot(diff.T, np.dot(inv_cov, diff))  # (x - mu_C)^T Σ_C^{-1} (x - mu_C)
                term2 = -0.5 * logdet  # -0.5 * log(|Σ_C|)
                term3 = np.log(self.priors[c])  # log(P(y = C))
                scores[i, idx] = term1 + term2 + term3 

        return np.argmax(scores, axis=1)  # Assign to the class with highest score
    def predict(self, X):
        X = self.normalize(X)
        return self.probability(X)  

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

model = QDAClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = np.mean(preds == y_test)
print(f"Accurcay: {accuracy}")
print(classification_report(y_test,preds))

Calculating probablities: 10it [00:13,  1.35s/it]

Accurcay: 0.9061666666666667
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1109
           1       0.97      0.95      0.96      1339
           2       0.91      0.93      0.92      1254
           3       0.88      0.89      0.88      1183
           4       0.94      0.90      0.92      1174
           5       0.96      0.79      0.86      1107
           6       0.97      0.94      0.96      1183
           7       0.95      0.87      0.91      1258
           8       0.74      0.90      0.81      1177
           9       0.83      0.93      0.88      1216

    accuracy                           0.91     12000
   macro avg       0.91      0.91      0.91     12000
weighted avg       0.91      0.91      0.91     12000



In [7]:
def evaluate_model(model,train, labels):
    X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = np.mean(preds == y_test)
    print(f"{type(model)} validation accuracy: {acc}")
    print(classification_report(y_test, preds))
    return model, acc
os.chdir("data")
dta = np.load("spam-data.npz")
os.chdir("..")
train = dta["training_data"]
labels = dta["training_labels"]
test = dta["test_data"]
evaluate_model(LDAClassifier(), train, labels)

100%|██████████| 2/2 [00:00<00:00, 333.17it/s]

<class '__main__.LDAClassifier'> validation accuracy: 0.8802395209580839
              precision    recall  f1-score   support

           0       0.87      0.96      0.92       569
           1       0.90      0.70      0.79       266

    accuracy                           0.88       835
   macro avg       0.89      0.83      0.85       835
weighted avg       0.88      0.88      0.88       835



(<__main__.LDAClassifier at 0x273ddf20e50>, 0.8802395209580839)